# C. Perform MintPy Time Series Analysis

*Author: Alex Lewandowski; Alaska Satellite Facility*

This notebook assumes that you have loaded a time series into MintPy and have prepared a custom configuration file.


**MintPy Workflow**
<p align="left">
  <img width="600" src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/MintPyWorkflow.jpg">
</p>     
<p style="text-align: center;">
    (Figure from Yunjun et al., 2019)
</p>

- Blue Ovals: correcting unwrapping errors and inverting for the raw phase time-series
- Green Ovals: correcting for noise from different sources to obtain the displacement time-series
- Dashed Ovals: Optional Steps

---
## 0. Import Required Software


In [ ]:
from pathlib import Path
import shutil
import sys

from ipyfilechooser import FileChooser
from mintpy.cli import view, tsview, plot_network, plot_transection
import mintpy.utils
import numpy as np
import opensarlab_lib as osl

current = Path("..").resolve()
sys.path.append(str(current))
import util.util as util

---
## 1. Select your project's custom config file

- This is located in your project's `MintPy` directory
- It is a text file named after your project
  - `path/to/MinPy/my_project.txt`

In [ ]:
recent_mintpy_path = Path.cwd() / '.recent_mintpy_config'

file_chooser_path = util.get_recent_mintpy_config_path()
if file_chooser_path:
    fc = FileChooser(path=file_chooser_path.parent, filename=file_chooser_path.name, select_default=True)
else:
    file_chooser_path = Path.home()
    fc = FileChooser(file_chooser_path, select_default=False)
    
print("Select your custom MintPy config file (MintPy/my_project_name.txt):")
display(fc)

In [ ]:
if Path(fc.selected) != file_chooser_path:
    util.write_recent_mintpy_config_path(Path(fc.selected))

config_path = Path(fc.selected)
mint_path = config_path.parent
inputs_path = mint_path / 'inputs'
ifgram_path = inputs_path / 'ifgramStack.h5'
plot_path = mint_path / 'plots'
velocity_path = mint_path/'velocity.h5'

---
## 2. Modify the Network

**Run the `modify_network` step**

- Identifies and excludes interferograms (i.e. affected by remaining coherence phase-unwrapping errors) before the network inversion
- Updates metadata with changes to your custom config

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep modify_network

---
## 3. Plot the interferogram network

Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- `ifgramStack_coherence_spatialAvg.txt`: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- `Network.pdf`: Displays the network of interferograms on time-baseline coordinates, colorcoded by avergae coherence of the interferograms. 
- `CoherenceMatrix.pdf` shows the avergae coherence pairs between all available pairs in the stack.

In [ ]:
%matplotlib inline
with osl.work_dir(mint_path):
    scp_args = f'{inputs_path}/ifgramStack.h5 --cmap-vlist 0.2 0.7 1.0'
    plot_network.main(scp_args.split())
    plots = ['bperpHistory.pdf', 'coherenceHistory.pdf', 'coherenceMatrix.pdf', 'network.pdf']
    for p in plots:
        if (mint_path/p).exists():
            (mint_path/p).rename(f'{plot_path}/{p}')

---
## 4. Set the Reference Point

**Run the `reference_point` step**

The interferometric phase is a relative observation by nature. The phases of each unwrapped interferogram are relative with respect to an arbitrary pixel. Therfore, we need to reference all interferograms to a common reference pixel.

The `reference_point` step selects a common reference pixel for the stack of interferograms. The default approach of MintPy is to choose a pixel with the highest spatial coherence in the stack. Other options include specifying the longitude and latitude of the desired reference pixel or the line and column number of the refence pixel. 

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_point

**Running the "reference_step" adds additional attributes "REF_X, REF_Y" and "REF_LON, REF_LAT" to the `ifgramStack.h5` file. To see the attributes of the file run `info.py`**

In [ ]:
!info.py $inputs_path/ifgramStack.h5 | egrep 'REF_'

---
## 5. Run a Quick Overview

**Run the `quick_overview` step**

- Assess possible groud deformation using the velocity from traditional interferogram stacking 
    - *reference: Zebker et al. (1997, JGR)*
- Assess distribution of phase unwrapping error from the number of interferogram triplets with non-zero integer ambiguity of closure phase 
    - *reference: T_int in Yunjun et al. (2019, CAGEO). Related to section 3.2, equation (8-9) and Fig. 3d-e.*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep quick_overview

---
## 6. Inverting the Small Baseline network

**Run the `invert_network` step**

- Invert the network of differential unwrapped interferograms to estimate the time-series of unwrapped phase with respect to a reference acquisition date
- By default mintpy selects the first acquisition
- The estimated time-series is converted to distance change from radar to target and is provided in meters.

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep invert_network

---
## 7. Correct for Tropospheric Propagation Delays (Optional)


<div class="alert alert-warning">
<font face="Calibri" size="5"><b><font color='rgba(200,0,0,0.2)'> <u>CDS is migrating data and current service is impacted</u></font></b></font>
<br/>
<font face="Calibri" size="4">   February 14, 2024
</font>
<br/>
<font face="Calibri" size="3"><a href="https://forum.ecmwf.int/t/a-new-cds-soon-to-be-launched-expect-some-disruptions/1607">https://forum.ecmwf.int/t/a-new-cds-soon-to-be-launched-expect-some-disruptions/1607</a>
    <br/><br/>
<i><b>Expect:</b>
    <ul>
<li>longer queues</li>
<li>requests being cancelled</li>
<li>temporary closure of queues</li>
<li>requests failing with unusual error messages</li>
<li>catalogue temporarily offline</li>
</font>
</div>

**Run the `correct_troposphere` step**

***This will be skipped if `mintpy.troposphericDelay.method` is set to "no" in your custom config***

- Requires configuring a `.cdsapirc` file for CDS access
  - Run the `CDS_Access.ipynb` (Set Up Climate Data Store Access) Notebook
- Uses ECMWF [ERA5 climate reanalysis pressure data](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset&keywords=((%20%22Product%20type:%20Reanalysis%22%20)%20AND%20(%20%22Provider:%20Copernicus%20C3S%22%20))&text=pressure)

In [ ]:
era5_path = mint_path/"ERA5"
timeseries_era5_path = mint_path/"timeseries_ERA5.h5"
inputs_era5_path = mint_path/"inputs/ERA5.h5"

for f in [timeseries_era5_path, inputs_era5_path]:
    try:
        f.unlink()
    except FileNotFoundError:
        pass
try:
    shutil.rmtree(era5_path)
except FileNotFoundError:
    pass

!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_troposphere

---
## 8. Correct for DEM Errors

**Run the `correct_topography` step**

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_topography

## 9. Calculate the Root Mean Square (RMS) of Residual Phase Time-Series for Each Acquisition

**Run the `residual_RMS` step**

- *reference: Yunjun et al. (2019, section 4.9 and 5.4)*
- To remove the long wavelength component in space, a phase ramp is removed for each acquisition
- Sets optimal reference date to date with min RMS
- Sets exclude dates (outliers) to dates with RMS > cutoff * median RMS (Median Absolute Deviation)

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep residual_RMS

## 10. Reference the Entire Time-Series to One Date in Time

**Run the `reference_date` step**

- *reference: Yunjun et al. (2019, section 4.9)*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_date

## 11. Estimate The Long-Term Velocity Rate

**Run the `velocity` step**

The timeseries file contains three datasets:
- the `time-series` dataset, which is the interferometric range change for each acquisition relative to the reference acquisition
- the `date` dataset, which contains the acquisition date for each acquisition
- the `bperp` dataset, which contains the timeseries of the perpendicular baseline 

The ground deformation caused by many geophysical or anthropogenic processes are linear at first order approximation. Therefore it is common to estimate the rate of the ground deformation which is the slope of linear fit to the time-series. 

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep velocity

**Plot the velocity map**

Note: The min/max values in this plot should be adjusted to your data set. 

The output of the following call will show a data range. You can run the cell once to collect this information and re-run it after adjustments are made.

- You should update `vmin` and `vmax` to match your data range
- The `vmin` and `vmax` values that you assign will be used for additional plots in this notebook
- If you wish to set 0 deformation to the center of the color scale (green), adjust your min/max values so they are symmetrical
    - For example, given a data range of `[-10.691118, 13.904866]`, appropriate symmetrical min/max values would be `[-14.0, 14.0]` 

In [ ]:
colormap_select = osl.select_parameter(
    [
        "Display red for uplift, and blue for subsidence",
        "Display blue for uplift, and red for subsidence"
    ]
)
display(colormap_select)

In [ ]:
%matplotlib inline
cmap = 'jet' if 'red for uplift' in colormap_select.value else 'jet_r'

vel_vmin, vel_vmax = util.get_mintpy_vmin_vmax(mint_path/'velocity.h5', 0.05)

# uncomment the following 2 lines to manually set the vmin and vmax values
# vel_vmin = -2.0
# vel_vmax = 2.0

scp_args = f'{mint_path}/velocity.h5 velocity -v {vel_vmin} {vel_vmax} --dpi 600 --figsize 15 15 --outfile {plot_path}/velocity.png --update --colormap {cmap}'
view.main(scp_args.split())

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
</div>

---
## 12. Geocode velocity.h5 in Preparation for Creating a velocity.kmz

**Run the `geocode` step**

- This is unnecessary for geocoded HyP3 data but would be needed for non-geocoded data

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep geocode

---
## 13. Create a kmz File

**Run the `google_earth` step**

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep google_earth

---
## 14. Plot the unwrapped inverted timeseries

In [ ]:
geotiff_path = mint_path / 'GeoTiffs'
disp_path = geotiff_path / 'displacement_maps'
unwrapped_path = disp_path / 'unwrapped'
unwrapped_path.mkdir(parents=True, exist_ok=True)
ts_demErr_path = list(mint_path.glob('timeseries*_demErr.h5'))[0]

In [ ]:
inv_vmin, inv_vmax = util.get_mintpy_vmin_vmax(ts_demErr_path, 0.05)

# uncomment the following 2 lines to manually set the vmin and vmax values
# inv_vmin = -2.0
# inv_vmax = 2.0

scp_args = f'{ts_demErr_path} --notitle --notick --noaxis -v {inv_vmin} {inv_vmax} --dpi 600 --figsize 15 15 --outfile {unwrapped_path}/unwrapped_inverted_ts.png'
view.main(scp_args.split())

---
## 15. Plot a Motion Transect

**Select two points to define a transect on an interactive plot**

In [ ]:
%matplotlib widget

vel_vmin, vel_vmax = util.get_mintpy_vmin_vmax(velocity_path, 0.05)

# uncomment the following 2 lines to manually set the vmin and vmax values
# vel_vmin = -2.0
# vel_vmax = 2.0

vel_vmin /= 100
vel_vmax /= 100

data, vel_info = mintpy.utils.readfile.read(velocity_path)
mask = np.ma.masked_where(data==0, data)
data = mask.filled(fill_value=np.nan)
line = osl.LineSelector(data, figsize=(9, 9), cmap='jet', vmin=vel_vmin, vmax=vel_vmax)

**Plot the selected motion transect**

In [ ]:
geotrans = (float(vel_info['X_FIRST'])), float(vel_info['X_STEP']) , 0.0, float(vel_info['Y_FIRST']), 0.0, float(vel_info['Y_STEP'])

def geolocation(x, y, geotrans):
    return [geotrans[0]+x*geotrans[1], geotrans[3]+y*geotrans[5]]

try:
    pnt_1 = geolocation(line.pnt1[0][0], line.pnt1[0][1], geotrans)
    pnt_2 = geolocation(line.pnt2[0][0], line.pnt2[0][1], geotrans)
    print(f"point 1: {pnt_1}")
    print(f"point 2: {pnt_2}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error may occur if a line was not selected.</text>'))

scp_args = f'{mint_path}/velocity.h5 -v {vel_vmin} {vel_vmax} --start-lalo {pnt_1[1]} {pnt_1[0]} --end-lalo {pnt_2[1]} {pnt_2[0]} --outfile x'

with osl.work_dir(plot_path):
    # let MintPy cleanup its old subplots 
    # this has been fixed in MintPy 1.5.3 but is required for 1.5.1
    plot_transection.main(scp_args.split()[:-2] + ['--noverbose', '--nodisplay'])
    # plot transection after removing old subplots
    %matplotlib inline
    plot_transection.main(scp_args.split())

---
## 16. Plot the Cumulative Displacement Map and Point Displacement Time Series

- Use the `Time` bar below the Cumulative Displacement Map to view displacements for different time periods
- Click on the Cumulative Displacement Map to select points for displaying Point Displacement Time-Series

In [ ]:
%matplotlib widget

tsview.main([str(ts_demErr_path),
             f'-d={mint_path}/inputs/geometryGeo.h5', 
             f'-o={mint_path}/displacement_ts', 
             f'--outfile={mint_path}/displacement_ts.pdf'])